In [1]:
# =============================================
# 1️⃣ Install / Import Packages
# =============================================
!pip install --pre azure-ai-projects>=2.0.0b1
!pip install azure-identity

import os
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient

# =============================================
# 2️⃣ Define UKEF Agent (Live Azure Option)
# =============================================
USE_AZURE_AGENT = True  # Set False to use simulated UKEF agent

if USE_AZURE_AGENT:
    myEndpoint = "https://phd-agent-ukef-resource.services.ai.azure.com/api/projects/phd_agent_ukef"
    project_client = AIProjectClient(
        endpoint=myEndpoint,
        credential=DefaultAzureCredential(),
    )
    myAgent = "UKEF1"
    agent = project_client.agents.get(agent_name=myAgent)
    print(f"Retrieved agent: {agent.name}")
    openai_client = project_client.get_openai_client()

    def ask_ukef_agent(question):
        try:
            resp = openai_client.responses.create(
                input=[{"role": "user", "content": question}],
                extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
            )
            answer = resp.output_text
            if not answer.strip():
                return "Simulated fallback: Coverage applies to low-risk projects up to £5M."
            return answer
        except Exception:
            return "Simulated fallback: Coverage applies to low-risk projects up to £5M."
else:
    def ask_ukef_agent(question):
        return "Simulated fallback: Coverage applies to low-risk projects up to £5M."

# =============================================
# 3️⃣ Define PhD Agent (Simulated / Local RAG)
# =============================================
def ask_phd_agent(question):
    folder = "data_phd"
    if not os.path.exists(folder):
        os.makedirs(folder)
        with open(f"{folder}/doc1.txt", "w") as f:
            f.write("This is a sample PhD methodology document covering experiments and analysis.")
    
    docs = [open(f"{folder}/{f}").read() for f in os.listdir(folder) if f.endswith(".txt")]
    context = "\n".join(docs)
    return f"PhD Agent simulated answer:\nContext:\n{context}\nQuestion:\n{question}"

# =============================================
# 4️⃣ Define Finance Agent (Simulated)
# =============================================
def ask_finance_agent(question):
    folder = "data_finance"
    if not os.path.exists(folder):
        os.makedirs(folder)
        with open(f"{folder}/doc1.txt", "w") as f:
            f.write("Finance Agent docs: budgets, risk analysis, and cash flows.")
    
    docs = [open(f"{folder}/{f}").read() for f in os.listdir(folder) if f.endswith(".txt")]
    context = "\n".join(docs)
    return f"Finance Agent simulated answer:\nContext:\n{context}\nQuestion:\n{question}"

# =============================================
# 5️⃣ Orchestrator Agent
# =============================================
def orchestrator_agent(question):
    """
    Orchestrator = agent itself
    Routes queries to child agents and aggregates responses
    """
    q = question.lower()
    if "ukef" in q or "coverage" in q or "mandate" in q:
        return ask_ukef_agent(question)
    elif "phd" in q or "research" in q or "methodology" in q:
        return ask_phd_agent(question)
    elif "finance" in q or "budget" in q:
        return ask_finance_agent(question)
    else:
        # Call all agents and aggregate
        resp1 = ask_ukef_agent(question)
        resp2 = ask_phd_agent(question)
        resp3 = ask_finance_agent(question)
        return f"{resp1}\n---\n{resp2}\n---\n{resp3}"

# =============================================
# 6️⃣ Example Queries / Demo
# =============================================
example_questions = [
    "Explain coverage rules for UKEF projects.",
    "Summarize key points of my PhD methodology.",
    "How does the No Net Cost mandate apply?",
    "What is the maximum project risk we can cover?",
    "Provide budget insights for low-risk projects."
]

for q in example_questions:
    print("Q:", q)
    print("A:", orchestrator_agent(q))
    print("-"*80)


Retrieved agent: UKEF1
Q: Explain coverage rules for UKEF projects.
A: Simulated fallback: Coverage applies to low-risk projects up to £5M.
--------------------------------------------------------------------------------
Q: Summarize key points of my PhD methodology.
A: PhD Agent simulated answer:
Context:
This is a sample PhD methodology document covering experiments and analysis.
Question:
Summarize key points of my PhD methodology.
--------------------------------------------------------------------------------
Q: How does the No Net Cost mandate apply?
A: The 'No Net Cost' mandate requires UKEF to operate without incurring losses to public funds, ensuring that premiums, recoveries, and other income cover liabilities over time (UKEF 'No Net Cost' mandate document).
--------------------------------------------------------------------------------
Q: What is the maximum project risk we can cover?
A: Simulated fallback: Coverage applies to low-risk projects up to £5M.
---
PhD Agent simu